In [1]:
from Bio import Entrez

In [2]:
# In order to import from the python file without hassle, we add the current
# directory to the python path
import sys; sys.path.append(".")

Let's let the nice folks at NCBI know who we are.

In [3]:
Entrez.email = "kt1451@student.uni-lj.si"

# Genome assembly

## Problem 1: Eulerian paths

In this exercise, we're going to look at the Eulerian path search you learned about in lectures. 

**a)**
Implement the `kmers` function in `helper_functions.py`. Look at the function docstrings for expected input paramters and return types. Your kmers function should be able to produce kmers of arbitrary size e.g. if we want 7-mers for the sequence *mesenchyme*, the function should return a list containing the following sequences: *mesench*, *esenchy*, *senchym*, *enchyme*. Your function should support arbitrary strides, e.g. in the previous example we used stride 1, while if we used the same string with stride 2, we would obtain the strings *mesench* and *senchym*.
**[5 points]**

**b)**
Implement the `assemble_genome` function in `helper_functions.py`. Look at the function docstrings for expected input paramters and return types.  Your genome assembly function should work in the following way:
1. For an input list of sequences, e.g. kmers, construct a DeBruijn graph as seen in the lectures.
2. Find all possible Euerlian paths through the graph, i.e. all possible paths which visit each edge exactly once. Your paths should all start from a source node with in-degree zero. In case no such node exists, you may use the first sequence in the list of input sequences as your starting point. 
3. Decode your obtained paths into sequences, and return a list (or set) of unique genome assemblies as strings.
**[10 points]**

**c)**
Modify your `assemble_genome` function in `helper_functions.py` to accept a list of input sequences in any order and still produce a correct reconstruction. An easy way to check this is to construct a list of kmers, and shuffle it. Your function should return the same results as in part b), where the kmers are ordered.
**[5 points]**

In [4]:
from helper_functions import kmers, assemble_genome

## Problem 2: Minimum fragment length

We will follow the idealized example from lectures where we have perfect coverage and each kmer is assumed to be sequenced exactly once. However, as we have seen in the lectures, some assemblies do not always produce unambiguous results. Sometimes more than one reconstruction is valid! In this exercise, we want to determine the minimum fragment length necessary for unambiguous reconstruction.

**TASK:** Simulate the sequencing process by running your `kmers` function on the SARS-CoV-2 spike protein nucleotide sequence. Next, find all the valid assemblies of your obtained kmers. You'll notice that there are no ambiguities for longer kmers e.g. 50-mers, but we run into them when generating shorter "reads" (kmers). What is the minimum fragment length where we can unambiguously reconstruct the initial spike protein nucleotide sequence? In other words, what is smallest k for k-mers where we obtain exactly one valid assembly from our `assemble_genome` function? Save your answer into the `minimum_fragment_length` variable. Write your observations into the `minimum_fragment_length_comments` variable.

**[10 points]**

In [5]:
import pickle
from os import path
from typing import Tuple, Generator, List

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def load(organism_id: str) -> SeqRecord:
    """Load the NCBI record, use cached files if possible."""
    if not path.exists(path.join("data", f"{organism_id}.pkl.gz")):
        with Entrez.efetch(db="nucleotide", rettype="gb", id=organism_id) as handle:
            record = SeqIO.read(handle, "gb")
            with open(path.join("data", f"{organism_id}.pkl.gz"), "wb") as f:
                pickle.dump(record, f)
    else:
        with open(path.join("data", f"{organism_id}.pkl.gz"), "rb") as f:
            record = pickle.load(f)

    return record

In [6]:

record=load("NC_045512")
record.features
for i in record.features:
    #print("new i")
    c=0
    if(i.type=="CDS"):
        #print(i)
        for aa in i.qualifiers.items():
            if ("gene" in aa[0] and aa[1][0]=="S") or "spike" in aa[1][0]:    
                c=1
                #print(i)
        if c==1:
            for aa in i.qualifiers.items():
                if "translation" in aa[0]:
                    s=aa[1][0]
                        #fin_acc_cove_vals_spike[acc_c]=aa[1][0]
                    print(s)


MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGR

In [7]:


def assemble_genome_modified(seqs, k=None, stride=None):
    
   
    allseqs = seqs
    pairs = dict()

    for x in range(1,len(allseqs)):
        if allseqs[x-1] in pairs.keys():
            pairs[allseqs[x-1]].append([allseqs[x],0])
        else:
            pairs[allseqs[x-1]]=[[allseqs[x],0]]
    
    totalnum=len(allseqs)-1
    
    from copy import deepcopy

    reslist = set()

    def rek(curitem, curpath, curdict, curcount):
        curcount = curcount + 1
        if(len(reslist)>1): 
            print("more than one")
            print("k that has more than one solution is:",k," so the min k-mer is:",k+1)
            return reslist
        if curcount == totalnum + 1:
            reslist.add(curpath)
            reslist
            return
        if curitem in curdict.keys():
            for xx in range(0,len(curdict[curitem])):
                if curdict[curitem][xx][1] != 1: # make sure not visited
                    nextdict = deepcopy(curdict)
                    nextdict[curitem][xx][1] = 1 # mark it visited
                    nextitem = nextdict[curitem][xx][0]
                    nextpath = curpath + nextitem[k - stride : ]
                    rek(nextitem, nextpath, nextdict, curcount)

    rek(allseqs[0], allseqs[0], deepcopy(pairs), 0)

    return reslist

In [10]:
i=50
while( i>1):
    k=i
    st=1
    kmersCov=kmers(s,k,st)
    kk=assemble_genome_modified(kmersCov,k,st)
    print(i)
    if(len(kk)>1):
        ress=kk
        print("The the min sure lenght of kmer is:",i+1)
        break
    i-=1
    
print("this has at least two possible solutiobs: ",ress)

50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than one
k that has more than one solution is: 4  so the min k-mer is: 5
more than

In [8]:
minimum_fragment_length = 5
minimum_fragment_length_comments = """
The min lenght where we do not have more than one solutions is 5
Here is what i did:
I made new function assemble_genome_modified same code logic as assemble_genome but with modifications so that in stops
making sequences once it makes 2 sequences than returns, that way i get faster to know if some k-mer has more than one 
solution
Than I ran the assemble_genome from k-mers from 50 to 2
So from 50 to 5 i had only 1 solution
When I ran it on 4 it found that it has more than one solutions
So the smalles number to run the assemble_genome and find one solution is 5.
If you wish you can run assemble_genome for k=4 but takes longer time and I do not reccomend it.

Explanation: The minimum_fragment_length is small compared to the lenght of the genome because there are many different 
elements ( letters) to the k-mers sequences and they are pretty much differently arranged so if the sequence legth ( kmer)is
a little bigger the lower the chance is to get multiple solutions to the sequence. If I used unly the letters {A,C,T,G} than
when creating k-mers there would most likely be more solutions so the minimum_fragment_length would probably be lardger.
"""

## Problem 3: Unrealistic assumptions

In this homework, we are assuming perfect coverage, which means we assume every kmer will be sequenced from the genome exactly once. We simulate this by running our kmers function on a known genome sequence.

In the real world, these assumptions would hold no water. Many things have to be taken into consideration. For example, biologists first prepare the tissue to be sequenced. Imagine someone is preparing to sequence some mouse tissue, but they forget to wear gloves and contaminate the sample with some of their own cells, and bacteria from their hands. Then the mouse sample may contain human and bacterial DNA instead of just mouse DNA. Next, the sequencing process splits the DNA molecules into short fragments at random, so we might have the same region sequenced twice, while some regions will be completely skipped. Finally, the sequencing process itself works something like this: for every nucleotide in a sequence there exists some colored reagent which binds to it. So every base, ACTG, is assigned its own color. Each nucleotide then goes under a large microscopic camera, which literally takes pictures of every nucleotide, one by one. Then, this sequence of colors is converted into a nucleotide sequence. However, sometimes reagents don't bind to the nucleotides perfectly, so this can cause ambiguities or errors in the reads.

**Question:** Write down at least **three** things we might encounter in the real world for which we have not accounted for by using kmers (you may use some issues I listed above, but think of at least one other thing we haven't mentioned yet). For each of the issues, think about how you would approach each problem. You don't need to write down a full guide, just a general overview of how we might solve each problem. Write your answer into the `unrealistic_assumptions` variable.
**[5 points]**

In [9]:
unrealistic_assumptions = """
Assumption 1: The lenght of the k-mers sequences is not the same unless ideal case, so we have to comapre "k-mer" samples 
with different lenghts which makes the algorithm a little more complicated, so we have to search for maches of segments with 
different lenght which is far more complicated, since if we brake the bigger segments on smalled we kind of lose information
as in k-mers we have more solutions. 
    As a solution to this could be a try to brake it to equal k-mers if its tehnically possible but even if it is as I
    metioned above we may get more solutions this way and we may not know the length of the sequence we are looking for 
    which is additional problem. Another more realistic thing to do is write algorithm to try to combine k-mers with
    different length, the result should have the same exact lenght (if we know it) of the genome, or strick to the length
    that we think it is. Also we must consider +- length ( i will talk this bellow) because of mutations

Assumption 2: Degradation of DNA, for example we want to solve a murder case or recostruct a DNA of an "ancient" specie
As time passes the DNA brakes so it makes it hard to find valid sequence some articles suggest DNA could last millions of 
years if preserved well: "That means that, under ideal conditions, 
DNA would last about 6.8 million years, after which all the bonds would be broken. But DNA would not be readable after 
about 1.5 million years, the researchers said." 
https://www.livescience.com/38150-dna-degradation-rate.html#:~:text=DNA%20degrades%20over%20time%2C%20and,on%20how%20well%20it's%20preserved.&text=That%20means%20that%2C%20under%20ideal,million%20years%2C%20the%20researchers%20said.

This is very interesting when it comes to recovering Neanderthal DNA :
https://en.wikipedia.org/wiki/Neanderthal_genome_project#:~:text=They%20demonstrated%20that%20Neanderthal%20genomic,specific%20sequences%20can%20be%20studied.

    So my idea of controbuting to solution of reconstruction "ancient" DNA is to compare all possible samples and try to 
    recreate some genome as much as possible, also we can use DNA of closer specie (human DNA in case of a Neanderthal) as a 
    guide of finding that solution since both have to be very similar, also the Assumption1 problems I metioned above apply here
    As of we wikipedia article i shared also there are problems with bacterial and Human DNA to the Neanderthal speciements
    I will mention solution to this problem in Assumprion 4( it is metioned in the description of this problem)
    A futuristic solution can be : if we have very well observed DNA of organism we might guess the DNA pretty much by the 
    form of the fosil, but this may not hold water since it is not proven working I dont state it as a fact just might be...
    Also for sequencing DNA of some specie can be used DNA of a specie that has a part of its DNA (example: we humans have
    part of Neanderthal DNA)
    I am pretty sure that there are better methods to this that I can google but I like to stick to my ideas!
    An example: EAGER: efficient ancient genome reconstruction 
    https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0918-z

Assumption 3: The DNA of different k-mers can be from another DNA molecule (on same specie), which adds the possibility 
of mutation like more or less genetic material or just changed amino acids, so in this case k-mers or some other test may 
classify it as different but it is only a molecule or small partion of molecules lets say that part of the finger was 
exposed to some radiation and when the dna was taken some dna of those exact molecules is mutated or lets say DNA to cancer cells...
in this case the methos would classify it as different.
    So what can we do is take multiple samples so eliminate the risk that just this molecule was modified or maybe damaged
    and pay attention to the majority of results ( here  are many machine learning methods or mathematical methods that 
    can be implemented ). So with using k-mers we can run tests on multiple samples... but as metioned we must consider
    using different methods of finding the genome as much as we can
    
Assumption 4: This is mentioned assumption from the problem description so I will not go much into details- the DNA sample
has DNA from other species or bacterias
    Solution to this could be after sampling all the data and putting a lot of work to compare it to the already known 
    genomes so that we make sure it is not DNA from another specie... Also we need more samples to confirm the same result
    because in the single sample if it has some DNA from different specie no enough to notice that it has would make a lot
    of problems...
    

"""

**Question:** In all of the cases we have talked about above, we pretend we are going into the genome assembly process blind. We have some DNA fragments about which we know nothing, so we have to perform assembly from the ground up. However, often times, we know something about our organisms. For example, for SARS-CoV-2, it was suspected that it was related to SARS and MERS. Think about how we could use existing genomes from related species to facilitate genome assembly. Think back to homework 2, where we looked at sequence alignment tools. Could we use some of that machinery to help with assembling the SARS-CoV-2 genome? Write down how you would approach genome assembly for SARS-CoV-2, known that it's closely related to SARS, or even the bat RaTG13. Write your answer into the `assembly_with_alignment` variable.
**[5 points]**

In [10]:
assembly_with_alignment = """
YES! I mentioned some of those things in the prevous part of the problem. It is always the best idea to compare the genome 
to a genome of a similar species. They should be very similar since "big mutations" do not survive easily. So lets dig into
more details:
    As solution could be if we try to create a genome that has similar parts of overlapping sequences and the lenghts should
    also be similar. Mathematically it is not a must rule for this ( probability 1) but using probability the genes should
    be similar. So lets say we get a part of SARS-CoV-2 genome so in the algorith we use to recreate that genome we can 
    place that part of the genome in a similar place of the lenght of the SARS and MERS genome, here is obvious that the
    length of that sequence can not be short ( i dont want to talk with numbers but lets say like 50-mers are OK for this)
    Also since it have to make the example similar to HW2, so that part of the sequence has to be similar to some part of
    the sequences of the SARS and MERS if not the same- we used a few methods to calculate that similarity. Also even 
    looking to the shape of the virus we can use that kind of calculations even before we genetically aprove they are similar
    To mention the some of the methods that we used in HW2-global_alignment, local-alignment, Hamming distance, also I am 
    sure that in science there are plenty of other methods.
    As conclusion: using the knowledge/data of similar species can help A LOT, but not fully since we must run separate 
    analisys for a different virus. 

"""